In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = '54DYXJOBER2F4X1GZYLKUN1BK5CN5TWVURNKYSAQ052D2M3V' # your Foursquare ID
CLIENT_SECRET = 'IBWPSNOADLPZXKWVHUGENR5DXH5E0I1BI2TGSXIVXY3OSI4L' # your Foursquare Secret
ACCESS_TOKEN = 'WG1DFCNLOYXHROW4EBKPT2SJZIQU1V21OP0O0YJ3QWIVIWUL' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 54DYXJOBER2F4X1GZYLKUN1BK5CN5TWVURNKYSAQ052D2M3V
CLIENT_SECRET:IBWPSNOADLPZXKWVHUGENR5DXH5E0I1BI2TGSXIVXY3OSI4L


In [3]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Los Angeles, CA', 'San Francisco, CA', 'Dallas, TX', 'Seattle, WA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d196941735") # Hospital ID
    results[city] = requests.get(url).json()

In [4]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of hospital in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of hospital in New York, NY =  266
Showing Top 100
Total number of hospital in Los Angeles, CA =  210
Showing Top 100
Total number of hospital in San Francisco, CA =  186
Showing Top 100
Total number of hospital in Dallas, TX =  198
Showing Top 100
Total number of hospital in Seattle, WA =  185
Showing Top 100


In [6]:
maps[cities[0]]

In [7]:
maps[cities[1]]

In [8]:
maps[cities[2]]

In [9]:
maps[cities[3]]

In [10]:
maps[cities[4]]